### Load preprocessed data

Run the script that downloads and processes the MovieLens data.
Uncomment it to run the download & processing script.

In [1]:
#!python ../src/download.py

In [2]:
import numpy as np
fh = np.load('data/dataset.npz')

# We have a bunch of feature columns and last column is the y-target
# Note pytorch is finicky about need int64 types
train_x = fh['train_x'].astype(np.int64)
train_y = fh['train_y']

# We've already split into train & test
test_x = fh['test_x'].astype(np.int64)
test_y = fh['test_y']


n_user = int(fh['n_user'])
n_item = int(fh['n_item'])
n_occu = int(fh['n_occu'])
n_rank = int(fh['n_ranks'])

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from abstract_model import AbstractModel

Now we add this new `total_variation` regularizer. Instead of regularizing the *norm* of vector, as we frequently do with L2 regularization, we penalize the difference in subsequent elements. This is useful in temporal models: you want day 0 close to day 1, but you don't care if day0 and day 1 deviate away from zero.  

In [4]:
def total_variation(array):
    return torch.sum(torch.abs(array[:, :-1] - array[:, 1:]))

In [16]:
import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl

from pytorch_lightning.loggers import TensorBoardLogger


def l2_regularize(array):
    loss = torch.sum(array ** 2.0)
    return loss


class MF(AbstractModel):
    def __init__(self, n_user, n_item, n_occu, n_rank, 
                 train_x, train_y, test_x, test_y, 
                 k=18, kt=2, c_vector=1.0, c_bias=1.0,
                 c_ut=1.0, c_temp=1.0, c_ovector=1.0,
                 batch_size=128):
        super().__init__()
        self.save_data(train_x, train_y, test_x, test_y)
        # These are simple hyperparameters
        self.k = k
        self.n_user = n_user
        self.n_item = n_item
        self.c_vector = c_vector
        self.c_ovector = c_ovector
        self.c_bias = c_bias
        self.batch_size = batch_size
        
        # NEW: regularization hyperparams
        self.c_ut = c_ut
        self.c_temp = c_temp
        
        # These are learned and fit by PyTorch
        self.user = nn.Embedding(n_user, k)
        self.item = nn.Embedding(n_item, k)
        self.bias_user = nn.Embedding(n_user, 1)
        self.bias_item = nn.Embedding(n_item, 1)
        self.bias = nn.Parameter(torch.ones(1))
        self.occu = nn.Embedding(n_occu, k)
        
        # NEW: temporal vectors
        self.user_temp = nn.Embedding(n_user, kt)
        self.temp = nn.Embedding(n_rank, kt)


    def forward(self, inputs):
        # This is the most import function in this script
        # These are the user indices, and correspond to "u" variable
        user_id = inputs[:, 0]
        # Item indices, correspond to the "i" variable
        item_id = inputs[:, 1]
        # vector user = p_u
        vector_user = self.user(user_id)
        # vector item = q_i
        vector_item = self.item(item_id)
        # this is a dot product & a user-item interaction: p_u * q_i
        ui_interaction = torch.sum(vector_user * vector_item, dim=1)
        # Pull out biases
        bias_user = self.bias_user(user_id).squeeze()
        bias_item = self.bias_item(item_id).squeeze()
        biases = (self.bias + bias_user + bias_item)
        # occupation-item interaction
        occu_id = inputs[:, 3]
        vector_occu = self.occu(occu_id)
        oi_interaction = torch.sum(vector_user * vector_occu, dim=1)
        
        # NEW: user-time interaction
        rank = inputs[:, 2]
        vector_user_temp = self.user_temp(user_id)
        vector_temp = self.temp(rank)
        ut_interaction = torch.sum(vector_user_temp * vector_temp, dim=1)
        
        prediction = ui_interaction + oi_interaction + ut_interaction + biases
        return prediction

    def likelihood(self, prediction, target):
        # MSE error between target = R_ui and prediction = p_u * q_i
        loss_mse = F.mse_loss(prediction, target.squeeze())
        return loss_mse
    
    
    def prior(self):
        # Add new regularization to the biases
        prior_bias_user =  l2_regularize(self.bias_user.weight) * self.c_bias
        prior_bias_item = l2_regularize(self.bias_item.weight) * self.c_bias
        prior_user =  l2_regularize(self.user.weight) * self.c_vector
        prior_item = l2_regularize(self.item.weight) * self.c_vector
        prior_occu = l2_regularize(self.occu.weight) * self.c_ovector
        
        # New: total variation regularization
        prior_ut = l2_regularize(self.user_temp.weight) * self.c_ut
        prior_tv = total_variation(self.temp.weight) * self.c_temp
        
        total = (prior_user + prior_item + prior_bias_user + prior_bias_item + prior_occu +
                 prior_ut + prior_tv)
        return total


model = MF(n_user, n_item, n_occu, n_rank, train_x, train_y, test_x, test_y)

# add a logger
logger = TensorBoardLogger("tb_logs", name="bottleneck_model")

# We could have turned on multiple GPUs here, for example
# trainer = pl.Trainer(gpus=8, precision=16)    
trainer = pl.Trainer(max_epochs=5,
                     reload_dataloaders_every_epoch=True,
                     logger=logger)    

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


### Train model

In [17]:
trainer.test(model)

--------------------------------------------------------------------------------
TEST RESULTS
{'test_loss': tensor(48.1952)}
--------------------------------------------------------------------------------



{'test_loss': 48.195167541503906}

#### Run model

In [18]:
trainer.fit(model)
trainer.test(model)


  | Name      | Type      | Params
----------------------------------------
0 | user      | Embedding | 108 K 
1 | item      | Embedding | 71 K  
2 | bias_user | Embedding | 6 K   
3 | bias_item | Embedding | 3 K   
4 | occu      | Embedding | 378   
5 | user_temp | Embedding | 12 K  
6 | temp      | Embedding | 4 K   


/opt/conda/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


--------------------------------------------------------------------------------
TEST RESULTS
{'test_loss': tensor(1.2546)}
--------------------------------------------------------------------------------



{'test_loss': 1.2546216249465942}